In [7]:
import os
from typing import List, Literal
from pydantic import BaseModel, Field


from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.prompts import MessagesPlaceholder
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.tools.render import format_tool_to_openai_function 
from langchain.agents import tool
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate

OPENAI_API_KEY=os.environ['OPENAI_API_KEY']

In [30]:
class ExibirEBI(BaseModel):
    title: str = Field(description="Título do EBI resumido em poucas palavras")
    base: str = Field(description="Texto base, com versículos e referências")
    description: str = Field(description="10 Perguntas de observação, interpretação e aplicação, listadas em ordem numérica")
    footer: str = Field(description="Versículos de reflexão final")

In [31]:
@tool(args_schema=ExibirEBI)
def exibir_ebi(**dict_info:ExibirEBI) -> dict:
   """Chame essa função para criar issues no Jira"""
   print(dict_info["title"])
   print(dict_info["description"])
   print(dict_info["footer"])
   return "exibido com sucesso"

In [32]:
exibir_ebi.args_schema.schema_json()

'{"title": "ExibirEBI", "type": "object", "properties": {"title": {"title": "Title", "description": "T\\u00edtulo do EBI resumido em poucas palavras", "type": "string"}, "base": {"title": "Base", "description": "Texto base, com vers\\u00edculos e refer\\u00eancias", "type": "string"}, "description": {"title": "Description", "description": "10 Perguntas de observa\\u00e7\\u00e3o, interpreta\\u00e7\\u00e3o e aplica\\u00e7\\u00e3o, listadas em ordem num\\u00e9rica", "type": "string"}, "footer": {"title": "Footer", "description": "Vers\\u00edculos de reflex\\u00e3o final", "type": "string"}}, "required": ["title", "base", "description", "footer"]}'

In [48]:
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.5, model="gpt-4o")

In [49]:
model_with_tool = model.bind(functions=[format_tool_to_openai_function(exibir_ebi)])

In [50]:

memory= ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [51]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """ Você é um especialista em teologia e em criação de estudo bíblico indutivo.
     Sua base bíblica é batista
     O estudo bíblico deve ser bem completo, com texto de referência, e com no mínimo 10 perguntas (Observação, interpretação e aplicação)
     Você cria estudo bíblicos voltados para jovens
     Você deve auxilixar o usuário a construir um EBI com base num texto bíblico
     Você deve sempre exibir o estudo bíblico
     você deve colocar o trecho do texto base no EBI 
     Caso o usuário solicite mudandas no EBI você deve executá-la e sempre exibir o estudo bíblico com a modificação
     """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [52]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model_with_tool | OpenAIFunctionsAgentOutputParser()

In [53]:
agent_executor= AgentExecutor(agent=agent_chain, tools=[exibir_ebi], verbose=True, memory=memory)

In [54]:
response=agent_executor.invoke({'input':"faça um estudo bíblico sobre o texto de lucas 12"})



> Entering new AgentExecutor chain...

Invoking: `exibir_ebi` with `{'title': 'Estudo Bíblico Indutivo: Lucas 12', 'base': 'Lucas 12:1-34 (NVI)\n\n1 Nesse meio tempo, ajuntou-se uma multidão de milhares de pessoas, a ponto de se atropelarem umas às outras. Jesus começou a falar, primeiramente aos seus discípulos, dizendo: "Cuidado com o fermento dos fariseus, que é a hipocrisia.\n2 Não há nada escondido que não venha a ser descoberto, ou oculto que não venha a ser conhecido.\n3 O que vocês disseram nas trevas será ouvido à luz do dia, e o que sussurraram aos ouvidos dentro de casa será proclamado dos telhados.\n4 "Eu digo a vocês, meus amigos: Não tenham medo dos que matam o corpo e depois nada mais podem fazer.\n5 Mas eu lhes mostrarei a quem vocês devem temer: Temam aquele que, depois de matar o corpo, tem poder para lançar no inferno. Sim, eu digo, a esse vocês devem temer.\n6 Não se vendem cinco pardais por duas moedinhas? Contudo, nenhum deles é esquecido por Deus.\n7 Até os cab

In [55]:
response=agent_executor.invoke({'input':"refaça a pergunta 5"})



> Entering new AgentExecutor chain...
Claro, aqui está a pergunta 5 reformulada:

### Estudo Bíblico Indutivo: Lucas 12

#### Texto Base: Lucas 12:1-34 (NVI)

```plaintext
1 Nesse meio tempo, ajuntou-se uma multidão de milhares de pessoas, a ponto de se atropelarem umas às outras. Jesus começou a falar, primeiramente aos seus discípulos, dizendo: "Cuidado com o fermento dos fariseus, que é a hipocrisia.
2 Não há nada escondido que não venha a ser descoberto, ou oculto que não venha a ser conhecido.
3 O que vocês disseram nas trevas será ouvido à luz do dia, e o que sussurraram aos ouvidos dentro de casa será proclamado dos telhados.
4 "Eu digo a vocês, meus amigos: Não tenham medo dos que matam o corpo e depois nada mais podem fazer.
5 Mas eu lhes mostrarei a quem vocês devem temer: Temam aquele que, depois de matar o corpo, tem poder para lançar no inferno. Sim, eu digo, a esse vocês devem temer.
6 Não se vendem cinco pardais por duas moedinhas? Contudo, nenhum deles é esquecido por